In [1]:
#%pip list

In [2]:
%cd ../..

c:\Users\tacke\OneDrive\Documents\GitHub\Modern-Time-Series-Forecasting-with-Python-2E-1


In [3]:
import numpy as np
import pandas as pd
import time
import plotly.express as px
import plotly.graph_objects as go
import os
import plotly.io as pio
pio.templates.default = "plotly_white"

from pathlib import Path
from tqdm.autonotebook import tqdm
import warnings
import humanize


from statsforecast.core import StatsForecast
from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import *
from statsforecast.models import (
    Naive,
    SeasonalNaive,
    HoltWinters,
    ETS,
    AutoETS,
    ARIMA,
    Theta,
    TBATS,
    MSTL

)

from functools import partial
from src.utils.ts_utils import forecast_bias_aggregate, forecast_bias_NIXTLA
from src.utils.general import LogTime
from src.utils import plotting_utils
from IPython.display import display, HTML
# %load_ext autoreload
# %autoreload 2
np.random.seed(42)
tqdm.pandas()

C:\Users\tacke\AppData\Local\Temp\ipykernel_17756\2788629501.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
if 'NIXTLA_ID_AS_COL' in os.environ:
    del os.environ['NIXTLA_ID_AS_COL']
os.environ['NIXTLA_ID_AS_COL'] = '1'

In [5]:
os.makedirs("imgs/chapter_8", exist_ok=True)
preprocessed = Path("data/london_smart_meters/preprocessed")

In [6]:
def format_plot(fig, legends = None, xlabel="Time", ylabel="Value", title=""):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t:  t.update(name = next(names)))
    fig.update_layout(
            autosize=False,
            width=900,
            height=500,
            title_text=title,
            title={
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            titlefont={
                "size": 20
            },
            legend_title = None,
            yaxis=dict(
                title_text=ylabel,
                titlefont=dict(size=12),
            ),
            xaxis=dict(
                title_text=xlabel,
                titlefont=dict(size=12),
            )
        )
    return fig

In [7]:
#Readin the missing value imputed and train test split data
try:
    train_df = pd.read_parquet(preprocessed/"selected_blocks_train_missing_imputed.parquet")
    val_df = pd.read_parquet(preprocessed/"selected_blocks_val_missing_imputed.parquet")
    test_df = pd.read_parquet(preprocessed/"selected_blocks_test_missing_imputed.parquet")

    print("Train Min and Max Date",train_df.timestamp.min(), train_df.timestamp.max())
    print("Val Min and Max Date",val_df.timestamp.min(), val_df.timestamp.max())
    print("Test Min and Max Date",test_df.timestamp.min(), test_df.timestamp.max())
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. Please make sure you have run 01-Feature Engineering.ipynb in Chapter06
    </div>
    """))
    
# # #Choosing a smaller backtesting window because of runtime issues
# backtesting = test_df[test_df.timestamp.between(pd.Timestamp("2014-01-01"),pd.Timestamp("2014-01-08"))]
# print("Backtesting DF Min and Max Date",backtesting.timestamp.min(), backtesting.timestamp.max())

Train Min and Max Date 2012-01-01 00:00:00 2013-12-31 23:30:00
Val Min and Max Date 2014-01-01 00:00:00 2014-01-31 23:30:00
Test Min and Max Date 2014-02-01 00:00:00 2014-02-27 23:30:00


In [8]:
len(train_df.LCLid.unique())

150

In [9]:
train_df.head()

,timestamp,LCLid,energy_consumption,frequency,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,2012-04-21 00:00:00,MAC000768,0.844,30min,32544,Std,ACORN-A,Affluent,block_1,NO_HOLIDAY,...,251,6.42,3.54,994.960022,3.79,3.64,rain,partly-cloudy-night,0.82,Partly Cloudy
1,2012-04-21 00:30:00,MAC000768,0.265,30min,32544,Std,ACORN-A,Affluent,block_1,NO_HOLIDAY,...,251,6.42,3.54,994.960022,3.79,3.64,rain,partly-cloudy-night,0.82,Partly Cloudy
2,2012-04-21 01:00:00,MAC000768,0.262,30min,32544,Std,ACORN-A,Affluent,block_1,NO_HOLIDAY,...,251,6.20,3.61,994.979980,3.67,3.42,rain,partly-cloudy-night,0.83,Partly Cloudy
3,2012-04-21 01:30:00,MAC000768,0.234,30min,32544,Std,ACORN-A,Affluent,block_1,NO_HOLIDAY,...,251,6.20,3.61,994.979980,3.67,3.42,rain,partly-cloudy-night,0.83,Partly Cloudy
4,2012-04-21 02:00:00,MAC000768,0.046,30min,32544,Std,ACORN-A,Affluent,block_1,NO_HOLIDAY,...,246,5.68,3.52,994.820007,3.15,3.25,rain,clear-night,0.86,Clear


## Running Baseline Forecast for all consumers

In [10]:
lcl_ids = sorted(train_df.LCLid.unique())

Here we are running the forecast one step ahead at a time. If current timestep is $t$, then we train till $t-1$ and predict for $t$. And to predict $t+1$, we train till $t$, and so on.

Running this kind of backtesting for a two months on half-hourly dataset for 400 households take too long to feasibly compute. So to assess one-step ahead forecast performance in test and validation periods, we have chosen two baselines(Naive and Seasonal Naive) which can easily and quickly be done using pandas.

### Naive Forecast

In [11]:
#from src.utils.ts_utils import darts_metrics_adapter

In [12]:
models =  [Naive(), 
                SeasonalNaive(season_length=48*7)
                ]

model_names = [model.__class__.__name__ for model in models]

sf = StatsForecast(
    models=models,
    freq='30min',
    n_jobs=-1,
)

Doing a little cleanup and dataprep before modeling. Easiest way to validate our 1-step ahead models is to do a cross validation.  As such, to evaluate the validation data, we will append both the train_df and val_df, and then use cross validation for backtesting. In NIXTLA, this is very easy to do.

When you do crossvalidation in NIXTLA, you will notice a "Cutff" column added.  This represents where the training data was "cut off".  In our example below, you will notice the cutoff is 1-step prior to the timestamp, showcasing the 1-step ahead forecast.

In [13]:
tr = train_df[["LCLid","timestamp","energy_consumption"]]
vl = val_df[["LCLid","timestamp","energy_consumption"]]
ts = test_df[["LCLid","timestamp","energy_consumption"]]

tr_vl = pd.concat([tr, vl]) # Creating the full training + validation dataset for cross validation
tr_vl_ts = pd.concat([tr, vl, ts]) # Creating the full training + validation + test dataset for cross validation

tr_vl['LCLid'] = tr_vl['LCLid'].astype(str)
tr_vl_ts['LCLid'] = tr_vl_ts['LCLid'].astype(str)

In [14]:
tr_vl.head()

,LCLid,timestamp,energy_consumption
0,MAC000768,2012-04-21 00:00:00,0.844
1,MAC000768,2012-04-21 00:30:00,0.265
2,MAC000768,2012-04-21 01:00:00,0.262
3,MAC000768,2012-04-21 01:30:00,0.234
4,MAC000768,2012-04-21 02:00:00,0.046


In [15]:
crossvalidation_val_df = sf.cross_validation(
    df = tr_vl,
    h = 1,
    step_size = 1,
    n_windows = len(vl.timestamp.unique()),
    id_col = 'LCLid',
    time_col = 'timestamp',
    target_col = 'energy_consumption',

  )

In [16]:
crossvalidation_val_df.head()

,LCLid,timestamp,cutoff,energy_consumption,Naive,SeasonalNaive
0,MAC000061,2014-01-01 00:00:00,2013-12-31 23:30:00,0.165,0.179,0.157
1,MAC000061,2014-01-01 00:30:00,2014-01-01 00:00:00,0.167,0.165,0.083
2,MAC000061,2014-01-01 01:00:00,2014-01-01 00:30:00,0.150,0.167,0.054
3,MAC000061,2014-01-01 01:30:00,2014-01-01 01:00:00,0.091,0.150,0.053
4,MAC000061,2014-01-01 02:00:00,2014-01-01 01:30:00,0.047,0.091,0.049


In [17]:
crossvalidation_test_df = sf.cross_validation(
    df = tr_vl_ts,
    h = 1,
    step_size = 1,
    n_windows = len(ts.timestamp.unique()),
    id_col = 'LCLid',
    time_col = 'timestamp',
    target_col = 'energy_consumption',

  )

In [18]:
crossvalidation_val_df.head(3)

,LCLid,timestamp,cutoff,energy_consumption,Naive,SeasonalNaive
0,MAC000061,2014-01-01 00:00:00,2013-12-31 23:30:00,0.165,0.179,0.157
1,MAC000061,2014-01-01 00:30:00,2014-01-01 00:00:00,0.167,0.165,0.083
2,MAC000061,2014-01-01 01:00:00,2014-01-01 00:30:00,0.150,0.167,0.054


In [19]:
crossvalidation_test_df.head(3)

,LCLid,timestamp,cutoff,energy_consumption,Naive,SeasonalNaive
0,MAC000061,2014-02-01 00:00:00,2014-01-31 23:30:00,0.066,0.069,0.057
1,MAC000061,2014-02-01 00:30:00,2014-02-01 00:00:00,0.063,0.066,0.058
2,MAC000061,2014-02-01 01:00:00,2014-02-01 00:30:00,0.040,0.063,0.060


In [20]:
fcst_mase = partial(mase, seasonality=1)
fcst_mase.__name__ = "mase"
forecast_bias_NIXTLA.__name__ = "forecast_bias"

baseline_val_metrics_df = evaluate(
                        df   = crossvalidation_val_df.drop(['cutoff'], axis =1 ), 
                        metrics  = [mse, mae, rmse, fcst_mase, forecast_bias_NIXTLA],
                        models  = model_names,
                        train_df  = tr_vl[['timestamp', 'LCLid', 'energy_consumption']],
                        id_col = 'LCLid',
                        time_col = 'timestamp',
                        target_col = 'energy_consumption'
                         )
baseline_val_metrics_df.head()

,LCLid,metric,Naive,SeasonalNaive
0,MAC000061,mse,0.003561,0.008076
1,MAC000062,mse,0.040691,0.053449
2,MAC000066,mse,0.011811,0.032999
3,MAC000086,mse,0.048976,0.047775
4,MAC000126,mse,0.022801,0.051587


In [21]:
baseline_val_metrics_df[(baseline_val_metrics_df.LCLid =='MAC000066') & (baseline_val_metrics_df.metric=='forecast_bias')]

,LCLid,metric,Naive,SeasonalNaive
602,MAC000066,forecast_bias,0.05743,17.725857


In [22]:
baseline_val_metrics_df_pivot = (baseline_val_metrics_df
    .melt(id_vars = ['LCLid','metric'], value_vars = model_names, var_name ='Algorithm', value_name='score')
    .pivot_table(index = ['LCLid','Algorithm'], columns = 'metric', values = 'score', observed = 'True')
).reset_index()
baseline_val_metrics_df_pivot.head(10)

metric,LCLid,Algorithm,forecast_bias,mae,mase,mse,rmse
0,MAC000061,Naive,0.059105,0.033290,0.954536,0.003561,0.059672
1,MAC000061,SeasonalNaive,2.830015,0.059912,1.717861,0.008076,0.089865
2,MAC000062,Naive,0.076747,0.088534,1.167550,0.040691,0.201719
3,MAC000062,SeasonalNaive,-0.360015,0.101974,1.344793,0.053449,0.231191
4,MAC000066,Naive,0.057430,0.043099,1.110772,0.011811,0.108680
5,MAC000066,SeasonalNaive,17.725857,0.089192,2.298688,0.032999,0.181657
6,MAC000086,Naive,-0.008482,0.167172,1.873062,0.048976,0.221305
7,MAC000086,SeasonalNaive,2.280819,0.155553,1.742879,0.047775,0.218575
8,MAC000126,Naive,0.071034,0.071154,1.075346,0.022801,0.151001
9,MAC000126,SeasonalNaive,2.204522,0.120750,1.824890,0.051587,0.227127


In [23]:
[model.__class__.__name__ for model in models]

['Naive', 'SeasonalNaive']

In [24]:
baseline_test_metrics_df =  evaluate(
                        df   = crossvalidation_test_df.drop(['cutoff'], axis =1 ), 
                        metrics  = [mse, mae, rmse, fcst_mase, forecast_bias_NIXTLA],
                        models  = model_names,
                        train_df  = tr_vl[['timestamp', 'LCLid', 'energy_consumption']],
                        id_col = 'LCLid',
                        time_col = 'timestamp',
                        target_col = 'energy_consumption'
                         )
baseline_test_metrics_df.head()

,LCLid,metric,Naive,SeasonalNaive
0,MAC000061,mse,0.003875,0.009974
1,MAC000062,mse,0.038202,0.062443
2,MAC000066,mse,0.001024,0.004306
3,MAC000086,mse,0.054223,0.046820
4,MAC000126,mse,0.010566,0.022787


In [25]:
baseline_test_metrics_df_pivot = (baseline_test_metrics_df
    .melt(id_vars = ['LCLid','metric'], value_vars = model_names, var_name ='Algorithm', value_name='score')
    .pivot_table(index = ['LCLid','Algorithm'], columns = 'metric', values = 'score', observed = 'True')
).reset_index()
baseline_test_metrics_df_pivot.head(10)


metric,LCLid,Algorithm,forecast_bias,mae,mase,mse,rmse
0,MAC000061,Naive,-0.054528,0.036842,1.056369,0.003875,0.062247
1,MAC000061,SeasonalNaive,-3.651271,0.069009,1.978709,0.009974,0.099869
2,MAC000062,Naive,0.003205,0.089600,1.181598,0.038202,0.195454
3,MAC000062,SeasonalNaive,2.938325,0.109424,1.443039,0.062443,0.249886
4,MAC000066,Naive,-0.141533,0.006462,0.166546,0.001024,0.031992
5,MAC000066,SeasonalNaive,-30.429426,0.026859,0.692213,0.004306,0.065620
6,MAC000086,Naive,0.009986,0.174187,1.951657,0.054223,0.232859
7,MAC000086,SeasonalNaive,-2.228803,0.151837,1.701244,0.046820,0.216378
8,MAC000126,Naive,0.013225,0.055279,0.835422,0.010566,0.102791
9,MAC000126,SeasonalNaive,5.490382,0.086149,1.301965,0.022787,0.150955


In [26]:
# Clean up
naive_pred_val_df = (crossvalidation_val_df
                     .rename(columns = {'y':'energy_consumption','Naive':'naive_predictions'})
                     .drop(['cutoff','SeasonalNaive'],axis=1).set_index('timestamp')
)

naive_pred_test_df = (crossvalidation_test_df
                     .rename(columns = {'y':'energy_consumption','Naive':'naive_predictions'})
                     .drop(['cutoff','SeasonalNaive'],axis=1).set_index('timestamp')
)

snaive_pred_val_df = (crossvalidation_val_df
                     .rename(columns = {'y':'energy_consumption','SeasonalNaive':'snaive_predictions'})
                     .drop(['cutoff','Naive'],axis=1).set_index('timestamp')
)

snaive_pred_test_df = (crossvalidation_test_df
                     .rename(columns = {'y':'energy_consumption','SeasonalNaive':'snaive_predictions'})
                     .drop(['cutoff','Naive'],axis=1).set_index('timestamp')
)

baseline_metrics_val_df = baseline_val_metrics_df_pivot.copy()
baseline_metrics_test_df = baseline_test_metrics_df_pivot.copy()
                            
                                                  

# Overall Metrics

For the overall metrics, we can use the same accuracy function used above, but remove the aggregation by LCLid.

In [27]:
from src.utils import ts_utils

In [28]:
overall_metrics_naive_val = {
    "MAE": ts_utils.mae(crossvalidation_val_df["energy_consumption"], crossvalidation_val_df["Naive"]),
    "MSE": ts_utils.mse(crossvalidation_val_df["energy_consumption"], crossvalidation_val_df["Naive"]),
    "meanMASE": baseline_val_metrics_df[baseline_val_metrics_df.metric =='mase']["Naive"].mean(),
    "Forecast Bias": ts_utils.forecast_bias_aggregate(crossvalidation_val_df["energy_consumption"], crossvalidation_val_df["Naive"])
}

overall_metrics_naive_val

{'MAE': 0.0881616,
 'MSE': 0.04498119,
 'meanMASE': 1.0899727,
 'Forecast Bias': -0.0029618898391831935}

In [29]:
overall_metrics_snaive_val = {
    "MAE": ts_utils.mae(crossvalidation_val_df["energy_consumption"], crossvalidation_val_df["SeasonalNaive"]),
    "MSE": ts_utils.mse(crossvalidation_val_df["energy_consumption"], crossvalidation_val_df["SeasonalNaive"]),
    "meanMASE": baseline_val_metrics_df[baseline_val_metrics_df.metric =='mase']["SeasonalNaive"].mean(),
    "Forecast Bias": ts_utils.forecast_bias_aggregate(crossvalidation_val_df["energy_consumption"], crossvalidation_val_df["SeasonalNaive"])
}
overall_metrics_snaive_val

{'MAE': 0.12919722,
 'MSE': 0.077654414,
 'meanMASE': 1.5819468,
 'Forecast Bias': -1.0015331232898155}

In [30]:
overall_metrics_naive_test = {
    "MAE": ts_utils.mae(crossvalidation_test_df["energy_consumption"], crossvalidation_test_df["Naive"]),
    "MSE": ts_utils.mse(crossvalidation_test_df["energy_consumption"], crossvalidation_test_df["Naive"]),
    "meanMASE": baseline_test_metrics_df[baseline_test_metrics_df.metric =='mase']["Naive"].mean(),
    "Forecast Bias": ts_utils.forecast_bias_aggregate(crossvalidation_test_df["energy_consumption"], crossvalidation_test_df["Naive"])
}

overall_metrics_naive_test

{'MAE': 0.08566107,
 'MSE': 0.044766486,
 'meanMASE': 1.0499331,
 'Forecast Bias': 0.017577425433739073}

In [31]:
overall_metrics_snaive_test = {
    "MAE": ts_utils.mae(crossvalidation_test_df["energy_consumption"], crossvalidation_test_df["SeasonalNaive"]),
    "MSE": ts_utils.mse(crossvalidation_test_df["energy_consumption"], crossvalidation_test_df["SeasonalNaive"]),
    "meanMASE": baseline_test_metrics_df[baseline_test_metrics_df.metric =='mase']["SeasonalNaive"].mean(),
    "Forecast Bias": ts_utils.forecast_bias_aggregate(crossvalidation_test_df["energy_consumption"], crossvalidation_test_df["SeasonalNaive"])
}
overall_metrics_snaive_test

{'MAE': 0.12169953,
 'MSE': 0.07150105,
 'meanMASE': 1.4867879,
 'Forecast Bias': 4.066730574245776}

## Evaluation of Baseline Forecast

In [32]:
agg_metric_val_df = pd.DataFrame([overall_metrics_naive_val, overall_metrics_snaive_val], index=["Naive","Seasonal Naive"])

agg_metric_val_df.style.format({"MAE": "{:.3f}", 
                          "MSE": "{:.3f}", 
                          "meanMASE": "{:.3f}", 
                          "Forecast Bias": "{:.2f}%"}).highlight_min(color='lightgreen')

,MAE,MSE,meanMASE,Forecast Bias
Naive,0.088,0.045,1.090,-0.00%
Seasonal Naive,0.129,0.078,1.582,-1.00%


In [33]:
agg_metric_test_df = pd.DataFrame([overall_metrics_naive_test, overall_metrics_snaive_test], index=["Naive","Seasonal Naive"])

agg_metric_test_df.style.format({"MAE": "{:.3f}", 
                          "MSE": "{:.3f}", 
                          "meanMASE": "{:.3f}", 
                          "Forecast Bias": "{:.2f}%"}).highlight_min(color='lightgreen')

,MAE,MSE,meanMASE,Forecast Bias
Naive,0.086,0.045,1.050,0.02%
Seasonal Naive,0.122,0.072,1.487,4.07%


In [34]:
baseline_val_metrics_df_pivot

metric,LCLid,Algorithm,forecast_bias,mae,mase,mse,rmse
0,MAC000061,Naive,0.059105,0.033290,0.954536,0.003561,0.059672
1,MAC000061,SeasonalNaive,2.830015,0.059912,1.717861,0.008076,0.089865
2,MAC000062,Naive,0.076747,0.088534,1.167550,0.040691,0.201719
3,MAC000062,SeasonalNaive,-0.360015,0.101974,1.344793,0.053449,0.231191
4,MAC000066,Naive,0.057430,0.043099,1.110772,0.011811,0.108680
...,...,...,...,...,...,...,...
295,MAC005463,SeasonalNaive,0.351350,0.095401,1.675904,0.028323,0.168296
296,MAC005521,Naive,0.077072,0.193729,1.465386,0.261627,0.511495
297,MAC005521,SeasonalNaive,6.771974,0.220915,1.671020,0.290943,0.539391
298,MAC005529,Naive,0.004668,0.053521,0.992287,0.019193,0.138538


In [35]:
fig = px.histogram(baseline_val_metrics_df_pivot, 
                   x="mase", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="MASE", ylabel="Probability Density", title="Distribution of MASE in the dataset")
fig.update_layout(xaxis_range=[0,3.2])
# fig.write_image("imgs/chapter_8/mase_dist.png")
fig.show()

In [36]:
fig = px.histogram(baseline_val_metrics_df_pivot, 
                   x="mae", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=100, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="MAE", ylabel="Probability Density", title="Distribution of MAE in the dataset")
# fig.write_image("imgs/chapter_8/mae_dist.png")
fig.update_layout(xaxis_range=[0,0.5])
fig.show()

In [37]:
fig = px.histogram(baseline_val_metrics_df_pivot, 
                   x="mse", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=500, 
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="MSE", ylabel="Probability Density", title="Distribution of MSE in the dataset")
fig.update_layout(xaxis_range=[0,0.6])
# fig.write_image("imgs/chapter_8/mse_dist.png")
fig.show()

In [38]:
baseline_val_metrics_df_pivot

metric,LCLid,Algorithm,forecast_bias,mae,mase,mse,rmse
0,MAC000061,Naive,0.059105,0.033290,0.954536,0.003561,0.059672
1,MAC000061,SeasonalNaive,2.830015,0.059912,1.717861,0.008076,0.089865
2,MAC000062,Naive,0.076747,0.088534,1.167550,0.040691,0.201719
3,MAC000062,SeasonalNaive,-0.360015,0.101974,1.344793,0.053449,0.231191
4,MAC000066,Naive,0.057430,0.043099,1.110772,0.011811,0.108680
...,...,...,...,...,...,...,...
295,MAC005463,SeasonalNaive,0.351350,0.095401,1.675904,0.028323,0.168296
296,MAC005521,Naive,0.077072,0.193729,1.465386,0.261627,0.511495
297,MAC005521,SeasonalNaive,6.771974,0.220915,1.671020,0.290943,0.539391
298,MAC005529,Naive,0.004668,0.053521,0.992287,0.019193,0.138538


In [39]:
fig = px.histogram(baseline_val_metrics_df_pivot, 
                   x="forecast_bias", 
                   color="Algorithm",
                   pattern_shape="Algorithm", 
                   marginal="box", 
                   nbins=250,
                   barmode="overlay",
                   histnorm="probability density")
fig = format_plot(fig, xlabel="Forecast Bias", ylabel="Probability Density", title="Distribution of Forecast Bias in the dataset")
fig.update_layout(xaxis_range=[-40,40])
# fig.write_image("imgs/chapter_8/bias_dist.png")
fig.show()

In [40]:
baseline_pred_val_df = naive_pred_val_df.reset_index().merge(snaive_pred_val_df.reset_index().drop(columns='energy_consumption'), on=['timestamp','LCLid'], how='outer')
baseline_pred_test_df = naive_pred_test_df.reset_index().merge(snaive_pred_test_df.reset_index().drop(columns='energy_consumption'), on=['timestamp','LCLid'], how='outer')

## Saving the Baseline Forecasts and Metrics

In [41]:
os.makedirs("data/london_smart_meters/output", exist_ok=True)
output = Path("data/london_smart_meters/output")

In [42]:
baseline_pred_val_df.to_pickle(output/"single_step_backtesting_baseline_prediction_val_df.pkl")
baseline_metrics_val_df.to_pickle(output/"single_step_backtesting_baseline_metrics_val_df.pkl")
agg_metric_val_df.to_pickle(output/"single_step_backtesting_baseline_aggregate_metrics_val.pkl")
baseline_pred_test_df.to_pickle(output/"single_step_backtesting_baseline_prediction_test_df.pkl")
baseline_metrics_test_df.to_pickle(output/"single_step_backtesting_baseline_metrics_test_df.pkl")
agg_metric_test_df.to_pickle(output/"single_step_backtesting_baseline_aggregate_metrics_test.pkl")